In [1]:

#  subrountine TREE( N, adj, root)
#  Subroutine finds all unique vertex sets T={v_0,v_1,...,v_n} in graph G rooted at v_0
#  such that (v_i, v_j) is an edge in G for some v_i and v_j in T
#  So there is a spanning tree of length n in G rooted at v_0 that covers T.
#
#  Input: N = total number of vertices in G
#            adj[ ] = adjaceny list defining G
#            root = label of rooted vertex
#  Output: T[ m ] = list of all connected vertex sets of size (m+1), m=0,1,2,...,N-1
#                K(m) = number of sets in T[m]
#
def TREE( N, adj, root):
    import numpy as np
    K=np.zeros(N)
    T=[ [] ]*N
    
    T[0]=[{root}]             # the root of the tree
    K[0]=len(T[0])          # by definition

    for nv in range(1,N):
        T[nv]=[ ]               # start connected set of (size=nv)  list with empty list

        for  subT in T[nv-1]:     # take each connected set  of (size=nv-1) subT
            S=subT
            
            for element in S:          # take each vertex in subT
                A=adj[int(element)]  # A is the set of  vertices that are adjacent to selected vertex in subT
                
                for a in A:         # take each vertex in A
                    X=S | {a}      # take the union ( X = subT U a) where a is vertex in adjacency set A
                    if len(X) > len(S):    #  X is a set in T[nv]
                        if X not in T[nv]:   #  add X to T[nv] if X has not been previously added
                            T[nv].append(X)
                            
        K[nv]=len(T[nv])      # calculate the total number of sets in T[nv]

    return T,K


In [2]:

# subroutine GROW_1(N,adj,T,K,Nstep)
# Subroutine finds the distribution of the paths generated from a root specified by T[0][0]
# in a graph G among the vertex-spanning sets in T[0], T[1], ... , T[N] after Nstep-1.
# Subroutine calculates the probabilities of the path containing 1,2,...,N vertices at each step.
#
#  Input: N = total number of vertices in G
#            adj[ ] = adjaceny list defining G
#            T[ m ] = list of all connected vertex sets of size (m+1), m=0,1,2,...,N-1
#            K(m) = number of sets in T[m]
#            Nstep = number of steps at which calculation is stopped
#
# Output: A[ T[m] ] =  total number of paths belonging to each vertex spanning set after Nstep-1
#              Total(n) =  total number of possible paths after n steps, for n=0,1, ... , Nstep-1
#              p(n) = probability vector for path containing 1,2,...,N vertices after n steps
#
# Other variables: Kmax = max{K(m)}  
#                              c(i, j, k) = the number of paths with vertex set T[ i, j ] ending at vertex k at n step
#                              cx(i, j, k) = same as c(i, j, k) at the next  (n+1) step
#
def GROW_1(N,adj,T,K,Nstep):
    import numpy as np
# np.arrays declarations:
    Kmax=int(np.max(K))
    A=np.zeros((N,Kmax))
    Total=np.zeros(Nstep)
    c=np.zeros((N,Kmax,N))
    cx=np.zeros((N,Kmax,N))
    p=np.zeros((Nstep,N))

# at ns=0, initial condition:
    c[0,0,0]=1.
    p[0,0]=1.
    Total[0]=1.

# at ns=1. after 1 step
    cx[0,0,0]=0.
    for xi in range(int(K[1])):   # path moves from root to adjacent vertex
        cx[1,xi,0]=0.                   # no paths ending at root vertex
        rx=T[1][xi]-T[0][0]           # adjacent vertex 
        for r in rx:
            cx[1,xi,int(r)]=1.          # path of length 2 ending on adjacent vertex
    p[1,0]=0.                             # no possibility of path of length 1
    p[1,1]=1.                             # all paths contain 2 vertices, or 1 edge
    Total[1]=np.sum(cx)
    c=cx.copy( )                        # update for next step ns+1=2
    cx=np.zeros((N,Kmax,N))

# at ns =2, ... , Nstep-1
    for ns in range(2,Nstep):
        Nx=min(Nstep,N)             # calculates only up to Nstep if Nstep<N
        for i in range(1,Nx):
            for j in range(int(K[i])):
                for k in range(N):     # calculations for cx[i, j, k]        
                    sum0=0.               # sum0 = contribution from T[i] same size sets
                    sum1=0.               # sum1 = contribution from T[i-1] smaller size sets

                    if k in T[i][j]:           # check to see if T[i][j][k] is a valid path specification 
                                                  #  if not then cx[i, j, k] = 0.0
                                                  
                        for a in adj[k]:                        # sum all contribution from T[i][j] with end vector 
                            sum0=sum0+c[i,j,int(a)]   #  in adj[k]
                        
                            if i >1:                                    # sum all contribution from T[i-1] for i>1
                                for js in range(int(K[i-1])):                                   # all T[i-1] are considered
                                    if (a in T[i-1][js]) & (T[i-1][js]|{k}==T[i][j]):  # require a in adj[k] to be in T[i-1][js]
                                        sum1=sum1+c[i-1,js,int(a)]                        # AND T[i-1][js] U {k} == T[i][j] 
                                           
                    cx[i,j,k]=sum0+sum1       # next time step cx[ i, j, k] = sum0 +sum1                        
    
        Total[ns]=np.sum(cx)        # total all possible paths at ns step                
        A=np.sum(cx,axis=2)       # the T[i] distribution at the last step
        p[ns]=np.sum(A,axis=1)/Total[ns]    # the probability vector at ns step
        c=cx.copy( )                                        # update for next step ns+1
        cx=np.zeros((N,Kmax,N))

    return  A, Total, p



In [3]:
N=12
adj=[[]]*N
adj[0]=[4,6]
adj[1]=[6,8]
adj[2]=[7,9]
adj[3]=[4,8]
adj[4]=[0,3,9]
adj[5]=[10,11]
adj[6]=[0,1,7]
adj[7]=[2,6,11]
adj[8]=[1,3]
adj[9]=[2,4,10]
adj[10]=[5,9]
adj[11]=[5,7]
root=0

In [4]:
T,K = TREE(N,adj,root)

In [5]:
csvData=[['i','K(i)','T(i,j),j=0,1,...,K(i)-1']]
for i in range(N):
    XX=[]
    for j in range(int(K[i])):
        XX.append(T[i][j])
    csvData.append([i,K[i],XX])

In [6]:
import csv
with open('TREE.csv','w') as csvFile:
    writer=csv.writer(csvFile)
    writer.writerows(csvData)
csvFile.close()

In [7]:
Nstep=50
A, Total, p = GROW_1(N,adj,T,K,Nstep)

In [8]:
csvData=[['n','[p0,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11]','# paths']]
for i in range(Nstep):
    XX=[]
    for x in range(N):
        XX.append(p[i,x])
    csvData.append([i,XX,Total[i]])


In [9]:
with open('GROW_1.csv','w') as csvFile:
    writer=csv.writer(csvFile)
    writer.writerows(csvData)
csvFile.close()

The data files were altered with a simple "find & replace" to get rid of unwanted notational glitches.